In [40]:
from dataclasses import dataclass
from enum import Enum
from pathlib import Path
from typing import List

import matplotlib.pyplot as plt
import numpy as np

from gps_accuracy.gps_accuracy import GpxResult
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [83]:
from gps_accuracy.gps_accuracy import GpxEvaluator
import pandas as pd

class InputType(Enum):
    Touch = 1
    TUI = 2
    
class Metaphor(Enum):
    Gesture = 1
    Joystick = 2
    Car =  3
    
@dataclass
class ReferenceTrack:
    track_id: int
    file: Path
    
    def __init__(self, file_path: Path):
        self.track_id = int(file_path.stem)
        self.file = file_path

@dataclass
class RecordedTrack:
    track_id: int
    user_id: int
    input_type: InputType
    metaphor: Metaphor
    file: Path
    result: GpxResult
    
    def __init__(self, file_path: Path):
        file_name = file_path.stem
        parts = file_name.split("_")
        self.track_id: int = int(parts[0])
        self.user_id: int = int(parts[1])
        self.input_type: InputType = InputType[parts[2]]
        self.metaphor: Metaphor = Metaphor[parts[3]]
        self.file: Path = file_path
    
    def evaluate(self, reference_track: ReferenceTrack):
        evaluator = GpxEvaluator(reference_track.file, self.file)
        self.result = evaluator.evaluate()
              
@dataclass
class TrackRepository:
    reference_tracks: dict
    recorded_tracks: List[RecordedTrack]
    
    def __init__(self):
        reference_track_list = [ReferenceTrack(track_file) for track_file in Path("reference_tracks").iterdir() if track_file.is_file()]
        self.reference_tracks = {track.track_id: track for track in reference_track_list}
        self.recorded_tracks = [RecordedTrack(track_file) for track_file in Path("recorded_tracks").iterdir() if track_file.is_file()]
        self._evaluate() 
    
    def _evaluate(self):
        for track in self.recorded_tracks:
            reference_track = self.reference_tracks[track.track_id]
            track.evaluate(reference_track)
        
    def get_by_track(self, track_id: int) -> List[RecordedTrack]:
        return [track for track in self.recorded_tracks if track.track_id == track_id]

    def get_by_user(self, user_id: int) -> List[RecordedTrack]:
        return [track for track in self.recorded_tracks if track.user_id == user_id]
    
    def get_by_input_type(self, input_type: InputType) -> List[RecordedTrack]:
        return [track for track in self.recorded_tracks if track.input_type == input_type]
    
    def get_by_metaphor(self, metaphor: Metaphor) -> List[RecordedTrack]:
        return [track for track in self.recorded_tracks if track.metaphor == metaphor]
    
    def get_all(self) -> List[RecordedTrack]:
        return self.recorded_tracks

class Category(Enum):
    InputType = 1

class ResultPlotter:
    def __init__(self):
        self.repo = TrackRepository()
        tracks = self.repo.get_by_user(1)
        data = {
            'Track': [track.track_id for track in tracks],
            'Input': [f"{track.input_type.name}_{track.metaphor.name}" for track in tracks],
            'InputType': [track.input_type.name for track in tracks],
            'Time': [track.result.time for track in tracks],
            'MeanError': [track.result.mean for track in tracks],
            'MedianError': [track.result.median for track in tracks]
        }
        self.data_frame = pd.DataFrame(data)
        
    def print_times(self, category: Category = None):
        col = "Input" if category is None else category.name
        pivot_df = self.data_frame.pivot_table(index="Track", columns=col, values="Time")
        print(pivot_df)
        
    def print_mean_errors(self, category: Category = None):
        col = "Input" if category is None else category.name
        pivot_df = self.data_frame.pivot_table(index="Track", columns=col, values="MeanError")
        print(pivot_df)
    
    def print_median_errors(self, category: Category = None):
        col = "Input" if category is None else category.name
        pivot_df = self.data_frame.pivot_table(index="Track", columns=col, values="MedianError")
        print(pivot_df)
    

In [82]:
plotter = ResultPlotter()
plotter.print_times()
plotter.print_times(Category.InputType)
plotter.print_mean_errors()
plotter.print_mean_errors(Category.InputType)
plotter.print_median_errors()
plotter.print_median_errors(Category.InputType)


Input  TUI_Car  TUI_Joystick  Touch_Gesture  Touch_Joystick
Track                                                      
1         59.0          73.0           82.0            50.0
2        145.0         127.0          154.0           122.0
3        212.0         216.0          173.0           170.0
4         88.0         113.0          130.0            95.0
InputType    TUI  Touch
Track                  
1           66.0   66.0
2          136.0  138.0
3          214.0  171.5
4          100.5  112.5
Input   TUI_Car  TUI_Joystick  Touch_Gesture  Touch_Joystick
Track                                                       
1      1.307045      1.353243       1.533747        1.619588
2      1.457662      1.451574       1.323020        1.907478
3      1.403549      1.281827       1.583566        1.931482
4      2.221321      1.555127       1.306705        1.822990
InputType       TUI     Touch
Track                        
1          1.330144  1.576667
2          1.454618  1.615249
3         